<a href="https://colab.research.google.com/github/hernansilva21/pnl-tp/blob/main/desafio2/Desafio_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Procesamiento de Lenguaje Natural - CEIA
# Desafío 2
## Alumno: Hernán Matías Silva

Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
Probar términos de interés y explicar similitudes en el espacio de embeddings.
Intentar plantear y probar tests de analogías.
Graficar los embeddings resultantes.
Sacar conclusiones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec


Compilación de literatura completa de dominio público en español.

In [ ]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./textos_dataset', os.F_OK) is False:
    if os.access('libros.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q libros.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


In [ ]:
!unzip -q './textos_dataset/libros.zip' -d './libros_dataset'

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup

# Ruta de la carpeta con los archivos HTML
carpeta_libros = './libros_dataset'

# Lista para almacenar los párrafos
parrafos = []

# Recorrer todos los archivos en la carpeta
for archivo in os.listdir(carpeta_libros):
    if archivo.endswith('.html'):
        ruta_archivo = os.path.join(carpeta_libros, archivo)
        with open(ruta_archivo, 'r', encoding='utf-8') as f:
            contenido = f.read()
            soup = BeautifulSoup(contenido, 'html.parser')

            # Extraer todos los párrafos
            for parrafo in soup.find_all('p'):
                texto_parrafo = parrafo.get_text(strip=True)
                parrafos.append(texto_parrafo)

# Crear un DataFrame a partir de la lista de párrafos
df = pd.DataFrame(parrafos, columns=['parrafo'])

# Mostrar los primeros registros del DataFrame
print(df.head())

# Guardar el DataFrame en un archivo CSV
df.to_csv('./textos_dataset/parrafos_libros.csv', index=False)


                                             parrafo
0                                                   
1  Voy a hablar de Granada, o mejor dicho, voy a ...
2  Esas ideas que, sin orden preconcebido, y pudi...
3  Los artistas de aguja y tijera saben perfectam...
4  Si un campesino os pregunta qué medios debe em...


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 207181


### 1 - Preprocesamiento

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-10-99a399a8b871>:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentence_tokens.append(text_to_word_sequence(row[0]))


In [ ]:
sentence_tokens[:2]

[[],
 ['voy',
  'a',
  'hablar',
  'de',
  'granada',
  'o',
  'mejor',
  'dicho',
  'voy',
  'a',
  'escribir',
  'sobre',
  'granada',
  'unos',
  'cuantos',
  'artículos',
  'para',
  'exponer',
  'ideas',
  'viejas',
  'con',
  'espíritu',
  'nuevo',
  'y',
  'acaso',
  'ideas',
  'nuevas',
  'con',
  'viejo',
  'espíritu',
  'pero',
  'desde',
  'el',
  'comienzo',
  'dése',
  'por',
  'sentado',
  'que',
  'mi',
  'intención',
  'no',
  'es',
  'cantar',
  'bellezas',
  'reales',
  'sino',
  'bellezas',
  'ideales',
  'imaginarias',
  'mi',
  'granada',
  'no',
  'es',
  'la',
  'de',
  'hoy',
  'es',
  'la',
  'que',
  'pudiera',
  'y',
  'debiera',
  'ser',
  'la',
  'que',
  'ignoro',
  'si',
  'algún',
  'día',
  'será',
  'que',
  'por',
  'grandes',
  'que',
  'sean',
  'nuestras',
  'esperanzas',
  'nuestra',
  'fe',
  'en',
  'la',
  'fuerza',
  'inconsciente',
  'de',
  'las',
  'cosas',
  'por',
  'tan',
  'torcidos',
  'caminos',
  'marchamos',
  'las',
  'personas',
 

### 2 - Crear los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 207181


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 62741


### 3 - Entrenar embeddings

In [ ]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 54873644.0
Loss after epoch 1: 22235796.0
Loss after epoch 2: 13073168.0
Loss after epoch 3: 12745496.0
Loss after epoch 4: 12469560.0
Loss after epoch 5: 12238448.0
Loss after epoch 6: 6581616.0
Loss after epoch 7: 0.0
Loss after epoch 8: 0.0
Loss after epoch 9: 0.0
Loss after epoch 10: 0.0
Loss after epoch 11: 0.0
Loss after epoch 12: 0.0
Loss after epoch 13: 0.0
Loss after epoch 14: 0.0
Loss after epoch 15: 0.0
Loss after epoch 16: 0.0
Loss after epoch 17: 0.0
Loss after epoch 18: 0.0
Loss after epoch 19: 0.0


(146759872, 207145900)

### 4 - Ensayar

In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["dinero"], topn=5)

[('dinero…', 0.46923142671585083),
 ('déficit', 0.4308391809463501),
 ('cuidarme', 0.4233459234237671),
 ('abasto', 0.42317262291908264),
 ('peaje', 0.42294105887413025)]

In [ ]:
# el método `get_vector` permite obtener los vectores:
vector_dinero = w2v_model.wv.get_vector("dinero")
print(vector_dinero)

[ 0.47693962 -0.2359217  -0.4619958  -0.1965238  -0.05965226  0.19737852
 -0.5186582   0.09628768 -0.01699479  0.15049896  0.14285971  0.38950148
 -0.06126163  0.10191393 -0.2715478   0.01027008  0.14782956  0.2878727
  0.1859976  -0.49405912  0.11728859 -0.03826934  0.00542426  0.05069257
 -0.02344627 -0.18388493  0.0009743   0.2750295  -0.34778526 -0.40347898
  0.40298226 -0.0314961  -0.30506614  0.15004283 -0.5473525  -0.15984342
 -0.04392413  0.23895332 -0.13792145  0.1123101   0.3040139   0.2616676
  0.15392826  0.37357607  0.23134856  0.23320746 -0.07914295  0.10464758
  0.15480562  0.14940627  0.12530026  0.667187    0.07622235  0.1262947
  0.35557047  0.4652827   0.25515345  0.17693734  0.02925587  0.04048055
  0.6250403   0.27927184 -0.0657535   0.18613794 -0.17349656  0.33965865
 -0.12616378 -0.25460857 -0.11995158 -0.20922893 -0.40453082  0.095162
  0.09580968 -0.45915177 -0.20849814 -0.3451046  -0.1639717   0.47181946
 -0.24309513 -0.001597   -0.13212147 -0.1777917  -0.1934

In [ ]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_dinero)

[('dinero', 1.0000001192092896),
 ('dinero…', 0.46923142671585083),
 ('déficit', 0.4308391809463501),
 ('cuidarme', 0.4233459234237671),
 ('abasto', 0.42317262291908264),
 ('peaje', 0.42294105887413025),
 ('adquirirlo', 0.4206572473049164),
 ('peseta', 0.41958722472190857),
 ('rédito', 0.41660770773887634),
 ('pagarlo', 0.4153023362159729)]

### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

In [ ]:
resultado = w2v_model.wv.most_similar(positive=['rey', 'mujer'], negative=['hombre'], topn=1)
print(resultado)